# 3. Create a Single Table

In [1]:
import pandas as pd
import sqlite3

conn = sqlite3.connect('../Data/online_shop.db')

In [2]:
# view the transactions table
transactions = pd.read_sql('SELECT * FROM transactions', conn)
transactions.head()

,customer,item_id,purchase_date
0,Ava,1011,4/1/23
1,Ava,1014,4/1/23
2,Ava,1015,4/15/23
3,Ava,1018,5/1/23
4,Ben,2345,4/15/23


In [3]:
# view the items table
items = pd.read_sql('SELECT * FROM items', conn)
items.head()

,item_id,item_description,price,category,rating
0,1011,Paint,$15.99,Arts & Crafts,3.5
1,1012,Crayons,$2.87,Arts & Crafts,4.7
2,1013,Markers,$2.50,Arts & Crafts,4.8
3,1014,Brush,$1.99,Arts & Crafts,4.2
4,1015,Paper,$22.49,Arts & Crafts,4.5


In [4]:
# merge two tables together
df = transactions.merge(items, how='left', on='item_id')
df

,customer,item_id,purchase_date,item_description,price,category,rating
0,Ava,1011,4/1/23,Paint,$15.99,Arts & Crafts,3.5
1,Ava,1014,4/1/23,Brush,$1.99,Arts & Crafts,4.2
2,Ava,1015,4/15/23,Paper,$22.49,Arts & Crafts,4.5
3,Ava,1018,5/1/23,Scissors,$3.50,Arts & Crafts,4.6
4,Ben,2345,4/15/23,Dog Food,$29.99,Pet Supplies,4.9
...,...,...,...,...,...,...,...
105,Jenny,1011,4/20/23,Paint,$15.99,Arts & Crafts,3.5
106,Jenny,1014,4/20/23,Brush,$1.99,Arts & Crafts,4.2
107,Jenny,1018,4/20/23,Scissors,$3.50,Arts & Crafts,4.6
108,Jenny,1012,4/20/23,Crayons,$2.87,Arts & Crafts,4.7


# 4. Prepare Rows for Modeling

Goal: Predict which cusotomers are most likely to buy dog food in june.  
Each row should contain data for one customer.  
y: june dog food purchases.  
x: aggregation based on april and may data.  

In [5]:
df

,customer,item_id,purchase_date,item_description,price,category,rating
0,Ava,1011,4/1/23,Paint,$15.99,Arts & Crafts,3.5
1,Ava,1014,4/1/23,Brush,$1.99,Arts & Crafts,4.2
2,Ava,1015,4/15/23,Paper,$22.49,Arts & Crafts,4.5
3,Ava,1018,5/1/23,Scissors,$3.50,Arts & Crafts,4.6
4,Ben,2345,4/15/23,Dog Food,$29.99,Pet Supplies,4.9
...,...,...,...,...,...,...,...
105,Jenny,1011,4/20/23,Paint,$15.99,Arts & Crafts,3.5
106,Jenny,1014,4/20/23,Brush,$1.99,Arts & Crafts,4.2
107,Jenny,1018,4/20/23,Scissors,$3.50,Arts & Crafts,4.6
108,Jenny,1012,4/20/23,Crayons,$2.87,Arts & Crafts,4.7


In [6]:
df.dtypes

customer            object
item_id              int64
purchase_date       object
item_description    object
price               object
category            object
rating              object
dtype: object

In [7]:
# convert data types to datetime and numeric fields
df.purchase_date = pd.to_datetime(df.purchase_date, format='%m/%d/%y')
df.price = pd.to_numeric(df.price.str.replace('$', ''))
df.rating = pd.to_numeric(df.rating)

In [8]:
df.dtypes

customer                    object
item_id                      int64
purchase_date       datetime64[ns]
item_description            object
price                      float64
category                    object
rating                     float64
dtype: object

In [9]:
# Create a subset of april and may data
df_april_may = df[df.purchase_date.dt.month < 6]
df_april_may.head()

,customer,item_id,purchase_date,item_description,price,category,rating
0,Ava,1011,2023-04-01,Paint,15.99,Arts & Crafts,3.5
1,Ava,1014,2023-04-01,Brush,1.99,Arts & Crafts,4.2
2,Ava,1015,2023-04-15,Paper,22.49,Arts & Crafts,4.5
3,Ava,1018,2023-05-01,Scissors,3.50,Arts & Crafts,4.6
4,Ben,2345,2023-04-15,Dog Food,29.99,Pet Supplies,4.9


In [10]:
# Create a subset of june data
df_june = df[df.purchase_date.dt.month == 6]
df_june.head()

,customer,item_id,purchase_date,item_description,price,category,rating
6,Ben,2345,2023-06-15,Dog Food,29.99,Pet Supplies,4.9
12,Chloe,2345,2023-06-06,Dog Food,29.99,Pet Supplies,4.9
24,Blake,2545,2023-06-10,Cat Food,25.55,Pet Supplies,4.2
34,Maxwell,5674,2023-06-02,Video Game,30.99,Games & Toys,4.1
36,Isabel,3811,2023-06-15,Socks,7.50,Apparel,3.7


In [11]:
# Create a column of june dog food purchases
dog_food_rows = df_june[df_june.item_description == 'Dog Food']

june_dog_food_purchases = (
    dog_food_rows
    .groupby('customer')['item_id']
    .count()
    .rename('june dog food purchases')
)

june_dog_food_purchases

customer
Aiden     1
Ben       1
Calvin    1
Chloe     1
Lia       1
Lily      1
Olivia    1
Name: june dog food purchases, dtype: int64

In [12]:
# How much did each customer spend in april and may?
total_spend = df_april_may.groupby('customer')['price'].sum().rename('total_spend')
total_spend

customer
Aiden       222.16
Ava          43.97
Ben          44.19
Bennett      27.73
Blake        25.55
Calvin       29.99
Chloe        36.33
Daniel       17.46
Evelyn       66.19
Gavin        39.47
Henry       112.42
Isabel        2.79
Jenny        49.34
Kate         83.25
Lia          78.95
Lily         69.31
Madeline    122.63
Margaret      7.99
Maxwell      78.31
Nolan        67.51
Olivia       68.03
Sophie        2.57
Name: total_spend, dtype: float64

In [13]:
# each row now represents a customer
model_df = (pd.concat([june_dog_food_purchases, total_spend], axis=1)
           .fillna(0).reset_index()
           .rename(columns={'index':'customer'})
           )

model_df

,customer,june dog food purchases,total_spend
0,Aiden,1.0,222.16
1,Ben,1.0,44.19
2,Calvin,1.0,29.99
3,Chloe,1.0,36.33
4,Lia,1.0,78.95
5,Lily,1.0,69.31
6,Olivia,1.0,68.03
7,Ava,0.0,43.97
8,Bennett,0.0,27.73
9,Blake,0.0,25.55


# Prepare Columns for Modeling

## a. Text to numeric (Dummy Variables)

In [17]:
model_df.head()

,customer,june dog food purchases,total_spend
0,Aiden,1.0,222.16
1,Ben,1.0,44.19
2,Calvin,1.0,29.99
3,Chloe,1.0,36.33
4,Lia,1.0,78.95


In [18]:
# Goal: change the category coolumn into numeric values
df_april_may.head()

,customer,item_id,purchase_date,item_description,price,category,rating
0,Ava,1011,2023-04-01,Paint,15.99,Arts & Crafts,3.5
1,Ava,1014,2023-04-01,Brush,1.99,Arts & Crafts,4.2
2,Ava,1015,2023-04-15,Paper,22.49,Arts & Crafts,4.5
3,Ava,1018,2023-05-01,Scissors,3.50,Arts & Crafts,4.6
4,Ben,2345,2023-04-15,Dog Food,29.99,Pet Supplies,4.9


In [20]:
# specify a column to get dummy variables
category_dummies = pd.get_dummies(df_april_may.category).astype('int')
category_dummies

,Apparel,Arts & Crafts,Games & Toys,Personal Care,Pet Supplies
0,0,1,0,0,0
1,0,1,0,0,0
2,0,1,0,0,0
3,0,1,0,0,0
4,0,0,0,0,1
...,...,...,...,...,...
105,0,1,0,0,0
106,0,1,0,0,0
107,0,1,0,0,0
108,0,1,0,0,0


In [21]:
# combine with customer data
pd.concat([df_april_may.customer, category_dummies], axis=1)

,customer,Apparel,Arts & Crafts,Games & Toys,Personal Care,Pet Supplies
0,Ava,0,1,0,0,0
1,Ava,0,1,0,0,0
2,Ava,0,1,0,0,0
3,Ava,0,1,0,0,0
4,Ben,0,0,0,0,1
...,...,...,...,...,...,...
105,Jenny,0,1,0,0,0
106,Jenny,0,1,0,0,0
107,Jenny,0,1,0,0,0
108,Jenny,0,1,0,0,0


In [22]:
# group by customer so each row is a single customer
categories = pd.concat([df_april_may.customer, category_dummies], axis=1).groupby('customer').sum().reset_index()
categories.head()

,customer,Apparel,Arts & Crafts,Games & Toys,Personal Care,Pet Supplies
0,Aiden,0,0,0,0,8
1,Ava,0,4,0,0,0
2,Ben,0,0,0,0,2
3,Bennett,0,5,0,0,0
4,Blake,0,0,0,0,1


In [23]:
# add categories to the model dataframe
model_df = model_df.merge(categories, how='left', on='customer')
model_df.head()

,customer,june dog food purchases,total_spend,Apparel,Arts & Crafts,Games & Toys,Personal Care,Pet Supplies
0,Aiden,1.0,222.16,0,0,0,0,8
1,Ben,1.0,44.19,0,0,0,0,2
2,Calvin,1.0,29.99,0,0,0,0,1
3,Chloe,1.0,36.33,3,2,0,0,0
4,Lia,1.0,78.95,2,0,0,0,1


## b. DateTime to numeric (Dummy Variables)

In [24]:
model_df.head()

,customer,june dog food purchases,total_spend,Apparel,Arts & Crafts,Games & Toys,Personal Care,Pet Supplies
0,Aiden,1.0,222.16,0,0,0,0,8
1,Ben,1.0,44.19,0,0,0,0,2
2,Calvin,1.0,29.99,0,0,0,0,1
3,Chloe,1.0,36.33,3,2,0,0,0
4,Lia,1.0,78.95,2,0,0,0,1


In [25]:
model_df.shape

(22, 8)

In [26]:
# View the dataframe with april / may behavior
df_april_may.head()

,customer,item_id,purchase_date,item_description,price,category,rating
0,Ava,1011,2023-04-01,Paint,15.99,Arts & Crafts,3.5
1,Ava,1014,2023-04-01,Brush,1.99,Arts & Crafts,4.2
2,Ava,1015,2023-04-15,Paper,22.49,Arts & Crafts,4.5
3,Ava,1018,2023-05-01,Scissors,3.50,Arts & Crafts,4.6
4,Ben,2345,2023-04-15,Dog Food,29.99,Pet Supplies,4.9


In [27]:
df_april_may.shape

(93, 7)

In [28]:
# goal: add oon a new oclumn based on date (number of days since last purchase)

In [29]:
# find the last purchase date
last_purchase = df_april_may.groupby('customer')['purchase_date'].max()
last_purchase.head()

customer
Aiden     2023-05-19
Ava       2023-05-01
Ben       2023-04-20
Bennett   2023-04-25
Blake     2023-05-10
Name: purchase_date, dtype: datetime64[ns]

In [30]:
# note down today's date
today = pd.Series(pd.to_datetime('2023-06-01'), index=last_purchase.index)
today.head()

customer
Aiden     2023-06-01
Ava       2023-06-01
Ben       2023-06-01
Bennett   2023-06-01
Blake     2023-06-01
dtype: datetime64[ns]

In [31]:
# find the number of days since the last purchase
days_between = (today - last_purchase).dt.days.rename('days_between')
days_between.head()

customer
Aiden      13
Ava        31
Ben        42
Bennett    37
Blake      22
Name: days_between, dtype: int64

In [32]:
# add the column to the model_df dataframe
model_df = model_df.merge(days_between.reset_index(), how='left', on='customer')
model_df.head()

,customer,june dog food purchases,total_spend,Apparel,Arts & Crafts,Games & Toys,Personal Care,Pet Supplies,days_between
0,Aiden,1.0,222.16,0,0,0,0,8,13
1,Ben,1.0,44.19,0,0,0,0,2,42
2,Calvin,1.0,29.99,0,0,0,0,1,16
3,Chloe,1.0,36.33,3,2,0,0,0,28
4,Lia,1.0,78.95,2,0,0,0,1,12
